In [1]:
# Header from RIT notebook
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
##Setting up Google sdk environment
import os 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/marcelkc/.config/gcloud/application_default_credentials.json' 
os.environ['GCLOUD_PROJECT'] = 'som-nero-phi-nigam-starr' #som-nero-phi-rit
%load_ext google.cloud.bigquery
##Setting up BQ API
from google.cloud import bigquery
client=bigquery.Client()
import numpy as np
import pandas as pd

/share/sw/open/anaconda/3/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [2]:
query = """
WITH
-- Collate patients receiving any recent acute respiratory illness (ARI) diagnosis
recentARI AS
( -- Look for any patient with an ARI, based on diagnosis code list suggested by CDC,
  --  that was recorded recently (since 12/2019).
  SELECT
    condOccur.person_id, visit_occurrence_id, condition_start_DATE, condition_start_DATETIME, 
    condition_source_value, condition_source_concept_id,
    visit_concept_id, care_site_name
  FROM
    `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.condition_occurrence` AS condOccur 
  LEFT JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.visit_occurrence` USING (visit_occurrence_id)
  LEFT JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.care_site` USING (care_site_id)
  WHERE
    condition_start_DATE >= '2019-12-01' AND
    ( condition_source_value LIKE 'J00.%' OR
      condition_source_value LIKE 'J01.%' OR
      condition_source_value LIKE 'J02.%' OR
      condition_source_value LIKE 'J03.%' OR
      condition_source_value LIKE 'J04.%' OR
      condition_source_value LIKE 'J05.%' OR
      condition_source_value LIKE 'J06.%' OR
      condition_source_value LIKE 'J09.%' OR
      condition_source_value LIKE 'J10.%' OR
      condition_source_value LIKE 'J11.%' OR
      condition_source_value LIKE 'J12.%' OR
      condition_source_value LIKE 'J13.%' OR
      condition_source_value LIKE 'J14.%' OR
      condition_source_value LIKE 'J15.%' OR
      condition_source_value LIKE 'J16.%' OR
      condition_source_value LIKE 'J17.%' OR
      condition_source_value LIKE 'J18.%' OR
      condition_source_value LIKE 'J20.%' OR
      condition_source_value LIKE 'J21.%' OR
      condition_source_value LIKE 'J22' OR
      condition_source_value LIKE 'J80' OR
      condition_source_value LIKE 'A37.91' OR
      condition_source_value LIKE 'A37.01' OR
      condition_source_value LIKE 'A37.11' OR
      condition_source_value LIKE 'A37.81' OR
      condition_source_value LIKE 'A48.1' OR
      condition_source_value LIKE 'B25.0' OR
      condition_source_value LIKE 'B44.0' OR
      condition_source_value LIKE 'B97.4' OR
      condition_source_value LIKE 'U07.1'
    )
),

firstARIInfo AS
(
SELECT person_id, firstARIDateTime, firstARI_visit_concept_id, firstARI_care_site_name
FROM (
  SELECT 
    person_id, 
    condition_start_DATETIME AS firstARIDateTime,
    visit_concept_id AS firstARI_visit_concept_id,
    care_site_name AS firstARI_care_site_name,
    row_number() OVER (partition by person_id ORDER BY condition_start_DATETIME ASC) AS row_num
   FROM recentARI
) AS r
WHERE r.row_num=1
),

lastARIInfo AS
(
SELECT person_id, lastARIDateTime, lastARI_visit_concept_id, lastARI_care_site_name
FROM (
  SELECT 
    person_id, 
    condition_start_DATETIME AS lastARIDateTime,
    visit_concept_id AS lastARI_visit_concept_id,
    care_site_name AS lastARI_care_site_name,
    row_number() OVER (partition by person_id ORDER BY condition_start_DATETIME DESC) AS row_num
   FROM recentARI
) AS r
WHERE r.row_num=1
),

recentARIDateTimeRange AS
(
SELECT person_id, lastARIDateTime, lastARI_visit_concept_id, lastARI_care_site_name,
firstARIDateTime, firstARI_visit_concept_id, firstARI_care_site_name
FROM firstARIInfo
INNER JOIN lastARIInfo USING (person_id)
)
,

-- recentARIDateTimeRange AS
-- ( -- Look for only the last / most recent ARI occurrence date for each patient, in case one has multiple
--   SELECT
--     recentARI.person_id,
--     MIN(condition_start_DATETIME) AS firstARIDateTime, MAX(condition_start_DATETIME) AS lastARIDateTime
--   FROM recentARI
--   GROUP BY recentARI.person_id 
-- ),


-- Collate results on anyone receiving the SARS-CoV2 NAA Test Results
resultsSARSCoV2Tests AS
(
  SELECT meas.person_id, visit_occurrence_id, measurement_DATE, measurement_DATETIME, value_source_value, value_source_value IN ('Detected','Pos','Positive') AS detectedSARSCoV2, visit_concept_id, care_site_name
  FROM `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.measurement` as meas
  LEFT JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.visit_occurrence` USING (visit_occurrence_id)
  LEFT JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.care_site` USING (care_site_id)
  WHERE measurement_concept_id = 706170 -- SARS-CoV2 NAA Test Result
  AND meas.value_source_value is NOT NULL -- Only capture usable results and not extra descriptors
) ,
/* Note how often get null values
select count(*) nRecords, count(distinct person_id) as nPerson, count(distinct visit_occurrence_id) as nVisits, countif(value_source_value is not null) as nonNullValues
from resultsSARSCoV2Tests 
nRecords  42268
nPerson 14934
nVisits 21043
nonNullValues 21197
*/  

-- time, concept_ids, care_site_name from first test
firstTestInfo AS
(
SELECT person_id, firstSARSCoV2TestedDateTime, firstTest_visit_concept_id, firstTest_care_site_name
FROM (
  SELECT 
    person_id, 
    measurement_DATETIME AS firstSARSCoV2TestedDateTime,
    visit_concept_id AS firstTest_visit_concept_id,
    care_site_name AS firstTest_care_site_name,
    row_number() OVER (partition by person_id ORDER BY measurement_DATETIME ASC) AS row_num
   FROM resultsSARSCoV2Tests
) AS r
WHERE r.row_num=1
),

-- time, concept_ids, care_site_name from last test
lastTestInfo AS
(
SELECT person_id, lastSARSCoV2TestedDateTime, lastTest_visit_concept_id, lastTest_care_site_name
FROM (
  SELECT 
    person_id, 
    measurement_DATETIME AS lastSARSCoV2TestedDateTime,
    visit_concept_id AS lastTest_visit_concept_id,
    care_site_name AS lastTest_care_site_name,
    row_number() OVER (partition by person_id ORDER BY measurement_DATETIME DESC) AS row_num
   FROM resultsSARSCoV2Tests
) AS r
WHERE r.row_num=1
),

resultsSARSCoV2DateTimeRange AS( -- Accounting for multiple possible tests per patient, group by patient ID and collate date time ranges
  SELECT 
    person_id, 
    firstSARSCoV2TestedDateTime, firstTest_visit_concept_id, firstTest_care_site_name,
    lastSARSCoV2TestedDateTime, lastTest_visit_concept_id, lastTest_care_site_name,
    firstSARSCoV2DetectedDateTime, lastSARSCoV2DetectedDateTime
  FROM firstTestInfo
  INNER JOIN lastTestInfo USING (person_id)
  LEFT JOIN -- Left outer join, because not everyone who had a test will have a positive test result to join to
  (
    SELECT person_id, MIN(measurement_DATETIME) AS firstSARSCoV2DetectedDateTime, MAX(measurement_DATETIME) AS lastSARSCoV2DetectedDateTime
    FROM resultsSARSCoV2Tests
    WHERE resultsSARSCoV2Tests.detectedSARSCoV2
    GROUP BY person_id 
  ) AS firstLastDetectedSARSCoV2Tests
  USING (person_id)
  ),

-- At this point:
--  recentARIDateTimeRange has one row per patient (41,341 rows as of 2020-04-20 cdm_release_DATE)
--  resultsSARSCoV2DateTimeRange has one row per patient (14,838)
-- Each with first and last dates of occurrences of ARI (after initial "recent" cutoff date) orSARS-CoV2 test results (including timing of positive/detected result, null if no positive/detected results)
recentARIandSARSCoV2DateTimeRange AS
( -- Full outer join in both directions because some people with ARI diagnosis don't get SARS-CoV2 testing and vice versa
  SELECT *,
      DATETIME_DIFF( firstSARSCoV2TestedDateTime, firstARIDateTime, DAY ) daysFirstARItoFirstSARSCoV2Tested,
      DATETIME_DIFF( firstSARSCoV2TestedDateTime, lastARIDateTime, DAY ) daysLastARItoFirstSARSCoV2Tested,
      DATETIME_DIFF( lastSARSCoV2TestedDateTime, firstARIDateTime, DAY ) daysFirstARItoLastSARSCoV2Tested,
      DATETIME_DIFF( lastSARSCoV2TestedDateTime, lastARIDateTime, DAY ) daysLastARItoLastSARSCoV2Tested,

      DATETIME_DIFF( firstSARSCoV2DetectedDateTime, firstARIDateTime, DAY ) daysFirstARItoFirstSARSCoV2Detected,
      DATETIME_DIFF( firstSARSCoV2DetectedDateTime, lastARIDateTime, DAY ) daysLastARItoFirstSARSCoV2Detected,
      DATETIME_DIFF( lastSARSCoV2DetectedDateTime, firstARIDateTime, DAY ) daysFirstARItoLastSARSCoV2Detected,
      DATETIME_DIFF( lastSARSCoV2DetectedDateTime, lastARIDateTime, DAY ) daysLastARItoLastSARSCoV2Detected,
  FROM 
    recentARIDateTimeRange FULL JOIN 
    resultsSARSCoV2DateTimeRange USING (person_id)
  -- 4,091 inner join results (recent ARI diagnosis code AND SARS-CoV2 test results exist, but not necessarily in the correct datetime order)
  -- 52,088 full outer join results (recent ARI diagnosi code OR SARS-CoV2 test results)
)

SELECT 
    recentARIandSARSCoV2DateTimeRange.*,
    year_of_birth, DATE_DIFF('2020-02-01', DATE(birth_DATETIME), YEAR) ageAsOfFeb2020,
    person.gender_concept_id , genderConc.concept_name AS genderConcept,
    person.race_source_value, person.race_concept_id, raceConc.concept_name as raceConcept,
    person.ethnicity_source_value, person.ethnicity_concept_id, ethnicityConc.concept_name as ethnicityConcept
  FROM 
    recentARIandSARSCoV2DateTimeRange 
    JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.person` AS person USING (person_id) 
    LEFT JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.concept` AS genderConc ON (person.gender_concept_id = genderConc.concept_id)
    LEFT JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.concept` AS raceConc ON (person.race_concept_id = raceConc.concept_id)
    LEFT JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.concept` AS ethnicityConc ON (person.ethnicity_concept_id = ethnicityConc.concept_id)
"""
query_job = client.query(query)
df = query_job.to_dataframe(progress_bar_type='tqdm')

Downloading: 100%|██████████| 52088/52088 [00:15<00:00, 3426.29rows/s]


In [3]:
df.columns

Index(['person_id', 'lastARIDateTime', 'lastARI_visit_concept_id',
       'lastARI_care_site_name', 'firstARIDateTime',
       'firstARI_visit_concept_id', 'firstARI_care_site_name',
       'firstSARSCoV2TestedDateTime', 'firstTest_visit_concept_id',
       'firstTest_care_site_name', 'lastSARSCoV2TestedDateTime',
       'lastTest_visit_concept_id', 'lastTest_care_site_name',
       'firstSARSCoV2DetectedDateTime', 'lastSARSCoV2DetectedDateTime',
       'daysFirstARItoFirstSARSCoV2Tested', 'daysLastARItoFirstSARSCoV2Tested',
       'daysFirstARItoLastSARSCoV2Tested', 'daysLastARItoLastSARSCoV2Tested',
       'daysFirstARItoFirstSARSCoV2Detected',
       'daysLastARItoFirstSARSCoV2Detected',
       'daysFirstARItoLastSARSCoV2Detected',
       'daysLastARItoLastSARSCoV2Detected', 'year_of_birth', 'ageAsOfFeb2020',
       'gender_concept_id', 'genderConcept', 'race_source_value',
       'race_concept_id', 'raceConcept', 'ethnicity_source_value',
       'ethnicity_concept_id', 'ethnicityCo

In [4]:
ARI = df['lastARIDateTime'].notnull().rename('ARI')
tested = df['lastSARSCoV2TestedDateTime'].notnull().rename('tested')
detected = df['lastSARSCoV2DetectedDateTime'].notnull().rename('detected')

in_time_range = lambda col : df[col].between(0, 60)
in_time_ranges = lambda col1, col2 : in_time_range(col1) | in_time_range(col2)

ARI_tested_60 = in_time_ranges('daysFirstARItoFirstSARSCoV2Tested', 'daysLastARItoFirstSARSCoV2Tested').rename('ARI_tested_60')
ARI_detected_60 = in_time_ranges('daysFirstARItoFirstSARSCoV2Detected', 'daysLastARItoFirstSARSCoV2Detected').rename('ARI_detected_60')

# age categorical
bins = [0, 5, 18, 50, 65, 75, 85, 100]
labels = ["[{:02d}, {:02d})".format(i, j) for i, j in zip(bins[:-1], bins[1:])]
age_cat = pd.cut(df['ageAsOfFeb2020'], bins, labels=labels).rename('age')

df['care_site'] = df['firstTest_care_site_name'].fillna(df['lastARI_care_site_name'])
df['visit_concept'] = df['firstTest_visit_concept_id'].fillna(df['lastARI_visit_concept_id'])

In [5]:
df_all = pd.concat([df, ARI, tested, detected, ARI_tested_60, ARI_detected_60, age_cat], axis=1)
df_all.head()

,person_id,lastARIDateTime,lastARI_visit_concept_id,lastARI_care_site_name,firstARIDateTime,firstARI_visit_concept_id,firstARI_care_site_name,firstSARSCoV2TestedDateTime,firstTest_visit_concept_id,firstTest_care_site_name,...,ethnicity_concept_id,ethnicityConcept,care_site,visit_concept,ARI,tested,detected,ARI_tested_60,ARI_detected_60,age
0,43715690,2020-01-23,9202.0,PEDIATRICS ALAMEDA PEDIATRICS,2020-01-23,9202.0,PEDIATRICS ALAMEDA PEDIATRICS,NaT,NaN,None,...,0,No matching concept,PEDIATRICS ALAMEDA PEDIATRICS,9202.0,True,False,False,False,False,"[05, 18)"
1,32150004,2019-12-30,9202.0,FAMILY MEDICINE ALMADEN SAN JOSE,2019-12-30,9202.0,FAMILY MEDICINE ALMADEN SAN JOSE,NaT,NaN,None,...,0,No matching concept,FAMILY MEDICINE ALMADEN SAN JOSE,9202.0,True,False,False,False,False,"[18, 50)"
2,30270769,2020-01-25,9202.0,FAMILY MEDICINE AFP ALAMEDA,2020-01-25,9202.0,FAMILY MEDICINE AFP ALAMEDA,NaT,NaN,None,...,0,No matching concept,FAMILY MEDICINE AFP ALAMEDA,9202.0,True,False,False,False,False,"[50, 65)"
3,30539784,2020-01-03,9202.0,FAMILY MEDICINE BVMG DANVILLE,2020-01-03,9202.0,FAMILY MEDICINE BVMG DANVILLE,NaT,NaN,None,...,0,No matching concept,FAMILY MEDICINE BVMG DANVILLE,9202.0,True,False,False,False,False,"[18, 50)"
4,42108785,2020-01-14,9202.0,FAMILY MEDICINE AFP ALAMEDA,2020-01-14,9202.0,FAMILY MEDICINE AFP ALAMEDA,NaT,NaN,None,...,0,No matching concept,FAMILY MEDICINE AFP ALAMEDA,9202.0,True,False,False,False,False,"[65, 75)"


In [6]:
print('sums')
for elt in ['ARI', 'tested', 'detected', 'ARI_tested_60', 'ARI_detected_60']:
    df_all[elt] = df_all[elt].astype('int32')
    print(elt, np.sum(df_all[elt]))

sums
ARI 41341
tested 14838
detected 1195
ARI_tested_60 3493
ARI_detected_60 249


In [15]:
print('person_id is unique')
df_all['person_id'].is_unique
df_all.isnull().sum()

person_id is unique


White |                                                                       12175
Asian |                                                                        4886
Other |                                                                        4393
|                                                                              4052
White | White or Caucasian                                                     3813
| Unknown                                                                      3229
| White or Caucasian                                                           2142
Unknown |                                                                      1548
Other | Other                                                                  1496
| Asian                                                                        1374
Asian | Asian                                                                  1353
| Other                                                                     

In [23]:
df_all['ethnicity_source_value'].value_counts().to_csv('ethnicity_source')
df_all['race_source_value'].value_counts().to_csv('race_source')
# for ind, grp in df_all.groupby('raceConcept'):
#     print(ind)
#     print(grp['race_source_value'].unique())
for ind, grp in df_all.groupby('ethnicityConcept'):
    print(ind)
    print(grp['ethnicity_source_value'].unique())

Hispanic or Latino
['Hispanic/Latino |' 'Hispanic/Latino | Not Hispanic'
 'Hispanic/Latino | Hispanic' 'Hispanic/Latino | Unknown'
 'Hispanic/Latino | Declines to State' 'Unknown | Hispanic' '| Hispanic'
 'Patient Refused | Hispanic']
No matching concept
['Unknown |' 'Patient Refused |' '| Unknown' 'Patient Refused | Unknown'
 '| Declines to State' '|' 'Unknown | Unknown'
 'Unknown | Declines to State' 'Patient Refused | Declines to State']
Not Hispanic or Latino
['Non-Hispanic/Non-Latino |' 'Non-Hispanic/Non-Latino | Not Hispanic'
 'Patient Refused | Not Hispanic' '| Not Hispanic'
 'Non-Hispanic/Non-Latino | Unknown' 'Non-Hispanic/Non-Latino | Hispanic'
 'Non-Hispanic/Non-Latino | Declines to State' 'Unknown | Not Hispanic']


In [8]:
visit_token = 'visit_concept'
from IPython.display import display, HTML
def df_disp(df):
    display(HTML(df.to_html()))

def format_by_visit(ind_nm):
    tbl = pd.pivot_table(df_all, values=['ARI', 'tested', 'detected', 'ARI_tested_60', 'ARI_detected_60'], index=[ind_nm],
                            columns=[visit_token], aggfunc=np.sum)
    p_tbl = tbl.T.rename(columns=str).reset_index()
    p_tbl = p_tbl.rename({'level_0' : 'type'}, axis=1).sort_values([visit_token, 'type'])

    cols = p_tbl.columns.tolist()
    cols = [cols[1], cols[0]] + cols[2:]
    p_tbl = p_tbl[cols]
    df_disp(p_tbl)
    p_tbl.to_csv('tbl1_visit_' + ind_nm + '.csv', index=False)
    return p_tbl
def format_by_ind(ind_nm):
    tbl = pd.pivot_table(df_all, values=['ARI', 'tested', 'detected', 'ARI_tested_60', 'ARI_detected_60'], index=[ind_nm],
                            aggfunc=np.sum)
    p_tbl = tbl.T
    df_disp(p_tbl)
    p_tbl.to_csv('tbl1_' + ind_nm + '.csv')
    return p_tbl

In [9]:
care_site_tok = 'care_site'
save_care_site_csv = lambda s, tok : s.value_counts().to_csv('care_site_counts_' + tok + '_visit.csv')
save_care_site_csv(df_all[care_site_tok], 'all')
save_care_site_csv(df_all[care_site_tok][df_all['ARI'] == 1], 'ARI')
save_care_site_csv(df_all[care_site_tok][df_all['tested'] == 1], 'tested')
save_care_site_csv(df_all[care_site_tok][df_all['detected'] == 1], 'detected')

In [10]:
for elt in ['age', 'ethnicityConcept', 'raceConcept', 'genderConcept']:
    format_by_visit(elt)
    format_by_ind(elt)

age,visit_concept,type,"[00, 05)","[05, 18)","[18, 50)","[50, 65)","[65, 75)","[75, 85)","[85, 100)"
0,0.0,ARI,1644.0,2382.0,2332.0,850.0,439.0,194.0,50.0
4,0.0,ARI_detected_60,0.0,4.0,73.0,27.0,15.0,6.0,0.0
8,0.0,ARI_tested_60,93.0,99.0,1195.0,373.0,159.0,50.0,11.0
12,0.0,detected,6.0,20.0,441.0,241.0,126.0,78.0,32.0
16,0.0,tested,364.0,445.0,5904.0,2273.0,1203.0,652.0,227.0
1,9201.0,ARI,1430.0,1229.0,2975.0,1206.0,660.0,438.0,133.0
5,9201.0,ARI_detected_60,0.0,2.0,40.0,26.0,8.0,11.0,5.0
9,9201.0,ARI_tested_60,53.0,50.0,417.0,229.0,114.0,77.0,22.0
13,9201.0,detected,0.0,3.0,73.0,39.0,15.0,23.0,5.0
17,9201.0,tested,108.0,103.0,950.0,626.0,316.0,214.0,54.0


age,"[00, 05)","[05, 18)","[18, 50)","[50, 65)","[65, 75)","[75, 85)","[85, 100)"
ARI,7976,9925,12871,5618,2856,1585,422
ARI_detected_60,0,7,127,68,25,17,5
ARI_tested_60,149,163,1869,785,330,151,37
detected,6,28,550,315,151,106,39
tested,485,592,7485,3347,1679,926,293


ethnicityConcept,visit_concept,type,Hispanic or Latino,No matching concept,Not Hispanic or Latino
0,0.0,ARI,981,1928,4997
4,0.0,ARI_detected_60,28,28,69
8,0.0,ARI_tested_60,301,335,1346
12,0.0,detected,117,607,220
16,0.0,tested,1121,4738,5224
1,9201.0,ARI,2169,96,5839
5,9201.0,ARI_detected_60,28,1,63
9,9201.0,ARI_tested_60,211,8,750
13,9201.0,detected,42,1,115
17,9201.0,tested,490,25,1872


ethnicityConcept,Hispanic or Latino,No matching concept,Not Hispanic or Latino
ARI,6304,7121,27916
ARI_detected_60,59,32,158
ARI_tested_60,570,419,2504
detected,174,617,404
tested,1783,4960,8095


raceConcept,visit_concept,type,American Indian or Alaska Native,Asian,Black or African American,Native Hawaiian or Other Pacific Islander,No matching concept,White
0,0.0,ARI,23.0,1354.0,238.0,75.0,3004.0,3212.0
4,0.0,ARI_detected_60,0.0,21.0,5.0,1.0,55.0,43.0
8,0.0,ARI_tested_60,5.0,342.0,62.0,24.0,643.0,906.0
12,0.0,detected,2.0,56.0,12.0,2.0,719.0,153.0
16,0.0,tested,22.0,1250.0,274.0,89.0,5948.0,3500.0
1,9201.0,ARI,22.0,1494.0,403.0,180.0,2330.0,3675.0
5,9201.0,ARI_detected_60,0.0,20.0,2.0,1.0,33.0,36.0
9,9201.0,ARI_tested_60,1.0,165.0,42.0,17.0,278.0,466.0
13,9201.0,detected,0.0,29.0,3.0,6.0,49.0,71.0
17,9201.0,tested,6.0,375.0,123.0,57.0,649.0,1177.0


raceConcept,American Indian or Alaska Native,Asian,Black or African American,Native Hawaiian or Other Pacific Islander,No matching concept,White
ARI,105,7760,1475,518,13412,18071
ARI_detected_60,0,45,10,3,92,99
ARI_tested_60,9,604,121,47,1050,1662
detected,2,100,20,10,790,273
tested,37,1861,441,160,6957,5382


genderConcept,visit_concept,type,FEMALE,MALE,No matching concept
0,0.0,ARI,4438.0,3465.0,3.0
4,0.0,ARI_detected_60,65.0,60.0,0.0
8,0.0,ARI_tested_60,1284.0,695.0,3.0
12,0.0,detected,453.0,491.0,0.0
16,0.0,tested,6751.0,4319.0,13.0
1,9201.0,ARI,4173.0,3931.0,NaN
5,9201.0,ARI_detected_60,36.0,56.0,NaN
9,9201.0,ARI_tested_60,446.0,523.0,NaN
13,9201.0,detected,62.0,96.0,NaN
17,9201.0,tested,1063.0,1324.0,NaN


genderConcept,FEMALE,MALE,No matching concept
ARI,22417,18918,6
ARI_detected_60,121,128,0
ARI_tested_60,2054,1436,3
detected,563,632,0
tested,8588,6237,13


In [11]:
for elt in ['age', 'ethnicity_source_value', 'race_source_value', 'genderConcept']:
    format_by_visit(elt)
    format_by_ind(elt)

age,visit_concept,type,"[00, 05)","[05, 18)","[18, 50)","[50, 65)","[65, 75)","[75, 85)","[85, 100)"
0,0.0,ARI,1644.0,2382.0,2332.0,850.0,439.0,194.0,50.0
4,0.0,ARI_detected_60,0.0,4.0,73.0,27.0,15.0,6.0,0.0
8,0.0,ARI_tested_60,93.0,99.0,1195.0,373.0,159.0,50.0,11.0
12,0.0,detected,6.0,20.0,441.0,241.0,126.0,78.0,32.0
16,0.0,tested,364.0,445.0,5904.0,2273.0,1203.0,652.0,227.0
1,9201.0,ARI,1430.0,1229.0,2975.0,1206.0,660.0,438.0,133.0
5,9201.0,ARI_detected_60,0.0,2.0,40.0,26.0,8.0,11.0,5.0
9,9201.0,ARI_tested_60,53.0,50.0,417.0,229.0,114.0,77.0,22.0
13,9201.0,detected,0.0,3.0,73.0,39.0,15.0,23.0,5.0
17,9201.0,tested,108.0,103.0,950.0,626.0,316.0,214.0,54.0


age,"[00, 05)","[05, 18)","[18, 50)","[50, 65)","[65, 75)","[75, 85)","[85, 100)"
ARI,7976,9925,12871,5618,2856,1585,422
ARI_detected_60,0,7,127,68,25,17,5
ARI_tested_60,149,163,1869,785,330,151,37
detected,6,28,550,315,151,106,39
tested,485,592,7485,3347,1679,926,293


ethnicity_source_value,visit_concept,type,Hispanic/Latino |,Hispanic/Latino | Declines to State,Hispanic/Latino | Hispanic,Hispanic/Latino | Not Hispanic,Hispanic/Latino | Unknown,Non-Hispanic/Non-Latino |,Non-Hispanic/Non-Latino | Declines to State,Non-Hispanic/Non-Latino | Hispanic,Non-Hispanic/Non-Latino | Not Hispanic,Non-Hispanic/Non-Latino | Unknown,Patient Refused |,Patient Refused | Declines to State,Patient Refused | Hispanic,Patient Refused | Not Hispanic,Patient Refused | Unknown,Unknown |,Unknown | Declines to State,Unknown | Hispanic,Unknown | Not Hispanic,Unknown | Unknown,|,| Declines to State,| Hispanic,| Not Hispanic,| Unknown
0,0.0,ARI,345.0,2.0,232.0,74.0,41.0,2160.0,50.0,59.0,996.0,305.0,114.0,6.0,2.0,13.0,14.0,221.0,7.0,9.0,100.0,68.0,305.0,223.0,276.0,1314.0,970.0
4,0.0,ARI_detected_60,18.0,0.0,10.0,0.0,0.0,53.0,0.0,0.0,13.0,1.0,1.0,0.0,0.0,0.0,1.0,6.0,0.0,0.0,2.0,0.0,20.0,0.0,0.0,0.0,0.0
8,0.0,ARI_tested_60,182.0,0.0,82.0,23.0,3.0,947.0,3.0,19.0,280.0,50.0,50.0,1.0,0.0,5.0,1.0,131.0,0.0,0.0,14.0,11.0,130.0,1.0,11.0,28.0,10.0
12,0.0,detected,87.0,0.0,26.0,1.0,2.0,176.0,1.0,0.0,31.0,4.0,3.0,0.0,0.0,0.0,1.0,51.0,0.0,0.0,6.0,1.0,551.0,0.0,1.0,2.0,0.0
16,0.0,tested,724.0,3.0,277.0,59.0,15.0,3818.0,20.0,56.0,938.0,168.0,151.0,1.0,5.0,19.0,4.0,667.0,2.0,6.0,72.0,29.0,3831.0,6.0,32.0,133.0,47.0
1,9201.0,ARI,910.0,6.0,986.0,110.0,109.0,4236.0,36.0,117.0,1060.0,304.0,49.0,2.0,NaN,7.0,3.0,33.0,NaN,1.0,8.0,1.0,3.0,NaN,47.0,71.0,5.0
5,9201.0,ARI_detected_60,21.0,0.0,6.0,0.0,1.0,60.0,0.0,1.0,2.0,0.0,1.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0
9,9201.0,ARI_tested_60,133.0,0.0,56.0,7.0,9.0,596.0,3.0,8.0,115.0,23.0,6.0,0.0,NaN,1.0,1.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,6.0,4.0,1.0
13,9201.0,detected,31.0,0.0,10.0,0.0,1.0,104.0,0.0,2.0,8.0,1.0,1.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0
17,9201.0,tested,326.0,1.0,121.0,11.0,13.0,1520.0,6.0,20.0,263.0,45.0,21.0,0.0,NaN,2.0,1.0,1.0,NaN,1.0,5.0,0.0,0.0,NaN,17.0,11.0,2.0


ethnicity_source_value,Hispanic/Latino |,Hispanic/Latino | Declines to State,Hispanic/Latino | Hispanic,Hispanic/Latino | Not Hispanic,Hispanic/Latino | Unknown,Non-Hispanic/Non-Latino |,Non-Hispanic/Non-Latino | Declines to State,Non-Hispanic/Non-Latino | Hispanic,Non-Hispanic/Non-Latino | Not Hispanic,Non-Hispanic/Non-Latino | Unknown,Patient Refused |,Patient Refused | Declines to State,Patient Refused | Hispanic,Patient Refused | Not Hispanic,Patient Refused | Unknown,Unknown |,Unknown | Declines to State,Unknown | Hispanic,Unknown | Not Hispanic,Unknown | Unknown,|,| Declines to State,| Hispanic,| Not Hispanic,| Unknown
ARI,2799,23,1743,412,301,16171,230,302,4954,1562,1015,20,24,148,68,1096,48,51,362,212,349,756,951,4187,3557
ARI_detected_60,42,0,16,0,1,134,0,1,18,3,2,0,0,0,1,9,0,0,2,0,20,0,0,0,0
ARI_tested_60,362,0,143,34,13,1876,7,28,454,84,87,1,0,7,3,171,0,1,16,15,130,1,17,32,11
detected,130,0,38,1,4,338,1,2,45,10,5,0,0,0,1,59,0,0,6,1,551,0,1,2,0
tested,1194,4,409,81,31,6153,27,81,1344,236,225,1,5,27,7,800,4,9,82,33,3835,6,50,145,49


race_source_value,visit_concept,type,American Indian or Alaska Native |,American Indian or Alaska Native | American Indian or Alaska Native,American Indian or Alaska Native | Asian,American Indian or Alaska Native | Other,American Indian or Alaska Native | Unknown,American Indian or Alaska Native | White or Caucasian,Asian - Historical Conv | Asian,Asian |,Asian | American Indian or Alaska Native,Asian | Asian,Asian | Black or African American,Asian | Declines to State,Asian | Native Hawaiian or Other Pacific Islander,Asian | Other,Asian | Unknown,Asian | White or Caucasian,"Asian, Hispanic |","Asian, non-Hispanic |","Asian, non-Hispanic | Asian","Asian, non-Hispanic | White or Caucasian",Black or African American |,Black or African American | American Indian or Alaska Native,Black or African American | Asian,Black or African American | Black or African American,Black or African American | Declines to State,Black or African American | Native Hawaiian or Other Pacific Islander,Black or African American | Other,Black or African American | Unknown,Black or African American | White or Caucasian,"Black, Hispanic |","Black, non-Hispanic |","Black, non-Hispanic | Black or African American","Native American, Hispanic |",Native Hawaiian or Other Pacific Islander |,Native Hawaiian or Other Pacific Islander | Asian,Native Hawaiian or Other Pacific Islander | Black or African American,Native Hawaiian or Other Pacific Islander | Declines to State,Native Hawaiian or Other Pacific Islander | Native Hawaiian or Other Pacific Islander,Native Hawaiian or Other Pacific Islander | Other,Native Hawaiian or Other Pacific Islander | Unknown,Native Hawaiian or Other Pacific Islander | White or Caucasian,Other |,Other | American Indian or Alaska Native,Other | Asian,Other | Black or African American,Other | Declines to State,Other | Native Hawaiian or Other Pacific Islander,Other | Other,Other | Unknown,Other | White or Caucasian,"Other, Hispanic |","Other, Hispanic | American Indian or Alaska Native","Other, Hispanic | Asian","Other, Hispanic | Black or African American","Other, Hispanic | Declines to State","Other, Hispanic | Native Hawaiian or Other Pacific Islander","Other, Hispanic | Other","Other, Hispanic | Unknown","Other, Hispanic | White or Caucasian","Other, non-Hispanic |","Other, non-Hispanic | Asian","Other, non-Hispanic | Other","Other, non-Hispanic | White or Caucasian","Pacific Islander, non-Hispanic |","Pacific Islander, non-Hispanic | Native Hawaiian or Other Pacific Islander",Patient Refused |,Patient Refused | American Indian or Alaska Native,Patient Refused | Asian,Patient Refused | Black or African American,Patient Refused | Declines to State,Patient Refused | Native Hawaiian or Other Pacific Islander,Patient Refused | Other,Patient Refused | Unknown,Patient Refused | White or Caucasian,Race and Ethnicity Unknown |,Race and Ethnicity Unknown | American Indian or Alaska Native,Race and Ethnicity Unknown | Asian,Race and Ethnicity Unknown | Black or African American,Race and Ethnicity Unknown | Declines to State,Race and Ethnicity Unknown | Other,Race and Ethnicity Unknown | Unknown,Race and Ethnicity Unknown | White or Caucasian,Unknown |,Unknown | American Indian or Alaska Native,Unknown | Asian,Unknown | Black or African American,Unknown | Declines to State,Unknown | Native Hawaiian or Other Pacific Islander,Unknown | Other,Unknown | Unknown,Unknown | White or Caucasian,White |,White | American Indian or Alaska Native,White | Asian,White | Black or African American,White | Declines to State,White | Native Hawaiian or Other Pacific Islander,White | Other,White | Unknown,White | White or Caucasian,"White, Hispanic |","White, Hispanic | Other","White, Hispanic | Unknown","White, Hispanic | White or Caucasian","White, non-Hispanic |","White, non-Hispanic | Declines to State","White, non-Hispanic | Other","White, non-Hispanic | Unknown","White, non-Hispanic | White or Caucasian",|,| American Indian or Alaska Native,| Asian,| Black o

race_source_value,American Indian or Alaska Native |,American Indian or Alaska Native | American Indian or Alaska Native,American Indian or Alaska Native | Asian,American Indian or Alaska Native | Other,American Indian or Alaska Native | Unknown,American Indian or Alaska Native | White or Caucasian,Asian - Historical Conv | Asian,Asian |,Asian | American Indian or Alaska Native,Asian | Asian,Asian | Black or African American,Asian | Declines to State,Asian | Native Hawaiian or Other Pacific Islander,Asian | Other,Asian | Unknown,Asian | White or Caucasian,"Asian, Hispanic |","Asian, non-Hispanic |","Asian, non-Hispanic | Asian","Asian, non-Hispanic | White or Caucasian",Black or African American |,Black or African American | American Indian or Alaska Native,Black or African American | Asian,Black or African American | Black or African American,Black or African American | Declines to State,Black or African American | Native Hawaiian or Other Pacific Islander,Black or African American | Other,Black or African American | Unknown,Black or African American | White or Caucasian,"Black, Hispanic |","Black, non-Hispanic |","Black, non-Hispanic | Black or African American","Native American, Hispanic |",Native Hawaiian or Other Pacific Islander |,Native Hawaiian or Other Pacific Islander | Asian,Native Hawaiian or Other Pacific Islander | Black or African American,Native Hawaiian or Other Pacific Islander | Declines to State,Native Hawaiian or Other Pacific Islander | Native Hawaiian or Other Pacific Islander,Native Hawaiian or Other Pacific Islander | Other,Native Hawaiian or Other Pacific Islander | Unknown,Native Hawaiian or Other Pacific Islander | White or Caucasian,Other |,Other | American Indian or Alaska Native,Other | Asian,Other | Black or African American,Other | Declines to State,Other | Native Hawaiian or Other Pacific Islander,Other | Other,Other | Unknown,Other | White or Caucasian,"Other, Hispanic |","Other, Hispanic | American Indian or Alaska Native","Other, Hispanic | Asian","Other, Hispanic | Black or African American","Other, Hispanic | Declines to State","Other, Hispanic | Native Hawaiian or Other Pacific Islander","Other, Hispanic | Other","Other, Hispanic | Unknown","Other, Hispanic | White or Caucasian","Other, non-Hispanic |","Other, non-Hispanic | Asian","Other, non-Hispanic | Other","Other, non-Hispanic | White or Caucasian","Pacific Islander, non-Hispanic |","Pacific Islander, non-Hispanic | Native Hawaiian or Other Pacific Islander",Patient Refused |,Patient Refused | American Indian or Alaska Native,Patient Refused | Asian,Patient Refused | Black or African American,Patient Refused | Declines to State,Patient Refused | Native Hawaiian or Other Pacific Islander,Patient Refused | Other,Patient Refused | Unknown,Patient Refused | White or Caucasian,Race and Ethnicity Unknown |,Race and Ethnicity Unknown | American Indian or Alaska Native,Race and Ethnicity Unknown | Asian,Race and Ethnicity Unknown | Black or African American,Race and Ethnicity Unknown | Declines to State,Race and Ethnicity Unknown | Other,Race and Ethnicity Unknown | Unknown,Race and Ethnicity Unknown | White or Caucasian,Unknown |,Unknown | American Indian or Alaska Native,Unknown | Asian,Unknown | Black or African American,Unknown | Declines to State,Unknown | Native Hawaiian or Other Pacific Islander,Unknown | Other,Unknown | Unknown,Unknown | White or Caucasian,White |,White | American Indian or Alaska Native,White | Asian,White | Black or African American,White | Declines to State,White | Native Hawaiian or Other Pacific Islander,White | Other,White | Unknown,White | White or Caucasian,"White, Hispanic |","White, Hispanic | Other","White, Hispanic | Unknown","White, Hispanic | White or Caucasian","White, non-Hispanic |","White, non-Hispanic | Declines to State","White, non-Hispanic | Other","White, non-Hispanic | Unknown","White, non-Hispanic | White or Caucasian",|,| American Indian or Alaska Native,| Asian,| Black or African American,

genderConcept,visit_concept,type,FEMALE,MALE,No matching concept
0,0.0,ARI,4438.0,3465.0,3.0
4,0.0,ARI_detected_60,65.0,60.0,0.0
8,0.0,ARI_tested_60,1284.0,695.0,3.0
12,0.0,detected,453.0,491.0,0.0
16,0.0,tested,6751.0,4319.0,13.0
1,9201.0,ARI,4173.0,3931.0,NaN
5,9201.0,ARI_detected_60,36.0,56.0,NaN
9,9201.0,ARI_tested_60,446.0,523.0,NaN
13,9201.0,detected,62.0,96.0,NaN
17,9201.0,tested,1063.0,1324.0,NaN


genderConcept,FEMALE,MALE,No matching concept
ARI,22417,18918,6
ARI_detected_60,121,128,0
ARI_tested_60,2054,1436,3
detected,563,632,0
tested,8588,6237,13
